In [3]:
import numpy as np
import cupy as cp
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qmatchatea.circuit.circuit import Qcircuit
from qmatchatea.preprocessing import preprocess
from qmatchatea.qk_utils import qk_transpilation_params
import qiskit.quantum_info as qi
import qiskit.circuit.library as qcl
from qiskit.extensions import UnitaryGate
from qtealeaves.observables import TNObsTensorProduct, TNObservables
import qtealeaves.observables as obs
from qmatchatea import QCOperators, QCConvergenceParameters, run_simulation, QCBackend
from qmatchatea.qk_utils import GHZ_qiskit, QFT_qiskit
from qmatchatea.py_emulator import run_py_simulation, QcMps
from tqdm.notebook import tqdm

# Example from qmatcha library

In [2]:
# Initialize operator class
operators = QCOperators()

# Define local observable
sigma_z = np.array([[1, 0], [0, -1]])
num_qubits = 10

# Add the observable operator to the list
operators.ops["sz"] = sigma_z

# Define Observable class
parity = TNObsTensorProduct("parity", "sz", num_qubits)

In [3]:
################################################################################
# We can then add the specific observable to the class :py:class:`TNObservables`,
# which is the one devoted to the input/output/measurement management.
# After doing that, it is time to set up the quantum circuit, which determines
# the evolution of the quantum state. For simplicity, we will
# use a function defined in the utility of this module, which creates a GHZ state.
# We recall that a GHZ state of :math:`n` qubits is defined as:
#
# .. math::
#   |GHZ\rangle = \frac{|0\rangle^{\otimes n}+|1\rangle^{\otimes n}}{\sqrt{2}}

# Add the observable to the IO class
observables = TNObservables()
observables += parity

# Initialize the quantum circuit and prepare a GHZ state
qc = QuantumCircuit(num_qubits)
_ = GHZ_qiskit(qc)

In [6]:
################################################################################
# We are now ready to run the simulation ( :doc:`/../chapters/interface` )!
# The function we use is :py:func:`run_simulation`. It really has a lot of
# customizations through different parameters. For this quickstart we will simply
# use the more basic parameters. Remember to pass the operators and the observables to the
# function. An important parameter to pass to this function is the **bond_dimension**,
# through the :py:class:`QCConvergenceParameters`, which underlines how much
# entanglement we can encode in our system. There is no way to know a priori which
# bond dimension is enough for your application. A way to discover it is to perform
# multiple simulation with different bond dimensions, and see after which value the
# results converges.
# For further information about the importance of the bond dimension see the
# example :doc:`random_quantum_circuit`.
# This time we are interested in the value of the Parity. However, the result
# class, :class:`simulation_results` contains many more interesting quantities.
# For further information about that, see :doc:`/../chapters/utils`
#
# It is possible to choose between different approaches for running the simulation:
# - The backend can be either python "PY" or fortran "FR".
# - The machine precision can be either double complex "Z" or single complex "C".
# - The device of the simulation can be either "cpu" or "gpu".
# - The number of processes for the MPI simulation. If 1 is passed, then a serial
#   program is run.
# - The approach for the MPI simulation. Either master/worker "MW", cartesian "CT" or
#   serial "SR".
#


backend = QCBackend(
    backend="PY", precision="Z", device="gpu", num_procs=1, mpi_approach="SR"
)


conv_params = QCConvergenceParameters(max_bond_dimension=10)
results = run_py_simulation(
    qc,
    convergence_parameters=conv_params,
    operators=operators,
    observables=observables,
    backend=backend,
)
observable_measures = results.observables

In [7]:
observable_measures

{'time': 0.07177591323852539,
 'energy': None,
 'norm': array(1.),
 'parity': (0.9999999999999998+0j),
 'projective_measurements': {}}

In [8]:
qc.draw()

┌───┐                                              ░ ┌─┐            »
    q_0: ┤ H ├──■───────────────────────────────────────────░─┤M├────────────»
         └───┘┌─┴─┐                                         ░ └╥┘┌─┐         »
    q_1: ─────┤ X ├──■──────────────────────────────────────░──╫─┤M├─────────»
              └───┘┌─┴─┐                                    ░  ║ └╥┘┌─┐      »
    q_2: ──────────┤ X ├──■─────────────────────────────────░──╫──╫─┤M├──────»
                   └───┘┌─┴─┐                               ░  ║  ║ └╥┘┌─┐   »
    q_3: ───────────────┤ X ├──■────────────────────────────░──╫──╫──╫─┤M├───»
                        └───┘┌─┴─┐                          ░  ║  ║  ║ └╥┘┌─┐»
    q_4: ────────────────────┤ X ├──■───────────────────────░──╫──╫──╫──╫─┤M├»
                             └───┘┌─┴─┐                     ░  ║  ║  ║  ║ └╥┘»
    q_5: ─────────────────────────┤ X ├──■──────────────────░──╫──╫──╫──╫──╫─»
                                  └───┘┌─┴─┐                ░  ║  ║  ║  ║  ║ »
    q_6: ──────────────────────────────┤ X ├──■─────────────░──╫──╫──╫──╫──╫─»
                                       └───┘┌─┴─┐           ░  ║  ║  ║  ║  ║ »
    q_7: ───────────────────────────────────┤ X ├──■────────░──╫──╫──╫──╫──╫─»
                                            └───┘┌─┴─┐      ░  ║  ║  ║  ║  ║ »
    q_8: ────────────────────────────────────────┤ X ├──■───░──╫──╫──╫──╫──╫─»
                                                 └───┘┌─┴─┐ ░  ║  ║  ║  ║  ║ »
    q_9: ─────────────────────────────────────────────┤ X ├─░──╫──╫──╫──╫──╫─»
                                                      └───┘ ░  ║  ║  ║  ║  ║ »
meas: 10/══════════════════════════════════════════════════════╩══╩══╩══╩══╩═»
                                                               0  1  2  3  4 »
«                        
«    q_0: ───────────────
«                        
«    q_1: ───────────────
«                        
«    q_2: ───────────────
«                        
«    q_3: ───────────────
«                        
«    q_4: ───────────────
«         ┌─┐            
«    q_5: ┤M├────────────
«         └╥┘┌─┐         
«    q_6: ─╫─┤M├─────────
«          ║ └╥┘┌─┐      
«    q_7: ─╫──╫─┤M├──────
«          ║  ║ └╥┘┌─┐   
«    q_8: ─╫──╫──╫─┤M├───
«          ║  ║  ║ └╥┘┌─┐
«    q_9: ─╫──╫──╫──╫─┤M├
«          ║  ║  ║  ║ └╥┘
«meas: 10/═╩══╩══╩══╩══╩═
«          5  6  7  8  9

# Try to construct our circuit

## Define H_perc

In [4]:
sigma_z  = np.diag([1., -1.])
identity = np.diag([1., 1.])

def kronecker_prod(operators):

    result = operators[0]
    for op in operators[1:]:

        result = np.kron(result, op)

    return result

def ReLU(x):
    return x * (x > 0)

def H_perc_nobatch(data, labels):
    n_data, n = data.shape

    h_perc = np.zeros((2**n, 2**n), dtype='float32')
    
    for i in tqdm(range(n_data), desc='Constructing H_perc'):

        op = np.zeros((2**n, 2**n), dtype='float32')
        for j in range(n):

            op += kronecker_prod([identity]*j+[data[i, j] * sigma_z]+[identity]*(n-j-1))

        h_perc += ReLU(-labels[i]*op)
        del op

    return (h_perc / np.sqrt(n)).astype('complex')

In [17]:
def H_perc_diag(data, labels):
    n_data, n = data.shape
    identity = np.ones((2,), 'float32')
    sigma_z  = np.array([1., -1.])

    h_perc = np.zeros((2**n,), dtype='float32')
    
    for i in tqdm(range(n_data), desc='Constructing H_perc'):

        op = np.zeros((2**n,), dtype='float32')
        for j in range(n):

            op += kronecker_prod([identity]*j+[data[i, j] * sigma_z]+[identity]*(n-j-1))

        h_perc += ReLU(-labels[i]*op)
        del op

    return (h_perc / np.sqrt(n)).astype('complex')

In [6]:
data = np.load('../data/patterns8_10.npy')
labels = np.ones((data.shape[0],))

h_perc = H_perc_nobatch(data[:,::1], labels)

Constructing H_perc:   0%|          | 0/8 [00:00<?, ?it/s]

In [35]:
eigvals, _ = np.linalg.eigh(h_perc)

In [38]:
np.allclose(np.sort(np.diag(h_perc)), eigvals)

True

In [8]:
h_perc_opt = H_perc_opt(data, labels)

Constructing H_perc:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
np.allclose(h_perc_opt, np.diag(h_perc))

True

In [11]:
# investigate solution
_, eigvecs = np.linalg.eigh(h_perc)
np.where(eigvecs[0])

(array([593]),)

In [13]:
N_data = 8
N_feat = 10
P = 100
dt = 0.5

num_qubits = N_feat


qc = QuantumCircuit(num_qubits)

# prepare initial state
for i in range(num_qubits):

    qc.h(i)

for p in tqdm(range(P), 'Constructing circuit'):
    frac  = (p+1)/P
    beta  = (1-frac)*dt
    gamma = frac*dt

    Uz = np.exp(-1.j*gamma*np.diag(h_perc), dtype='complex128')
    op = UnitaryGate(np.diag(Uz))

    # apply evolution with Uz (oracle)
    qc.append(op, range(num_qubits))

    # apply evoltion with Ux
    for i in range(num_qubits):

        qc.h(i)
        qc.p(-2*beta, i)
        qc.h(i)



#qc.draw()

Constructing circuit:   0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
qi.Operator(qc)

KeyboardInterrupt: 

In [8]:
# optimize circuit
qk_params = qk_transpilation_params(optimization=3, basis_gates=['x', 'y', 'z', 'h', 'cx', 'cy', 'cz', 'p', 'cp', 'r', 'cr', 'u1', 'u2', 'u3', 'cu', 'u', 'cnot'])
qc_opt = preprocess(qc, qk_params=qk_params)
qc_opt.depth()

KeyboardInterrupt: 

In [49]:
operators = QCOperators()
sigma_z = np.array([[1, 0], [0, -1]])
operators.ops["sz"] = sigma_z

In [6]:
observables = TNObservables()
observables += obs.TNState2File("state.txt", "F")
observables += obs.TNObsLocal('label', 'sx')

In [69]:
backend = QCBackend(
    backend="PY", precision="Z", device="gpu", num_procs=1, mpi_approach="SR"
)


conv_params = QCConvergenceParameters(max_bond_dimension=10)
results = run_py_simulation(
    qc_opt,
    convergence_parameters=conv_params,
    observables=observables,
    backend=backend,
)
observable_measures = results.observables

In [5]:
observable_measures

{'mps_state': [array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]]),
  array([[[-0.70710678+0.j],
          [-0.70710678+0.j]]])],
 'time': 1.0490984916687012,
 'energy': None,
 'norm': array(1.),
 'projective_measurements': {}}

## Using Qcircuit

In [6]:
qc = Qcircuit(num_qubits)
# prepare initial state
for i in range(num_qubits):

    qc.h(i)

for p in range(P):
    frac  = (p+1)/P
    beta  = (1-frac)*dt
    gamma = frac*dt

    Uz = np.exp(-1.j*gamma*np.diag(h_perc), dtype='complex128')
    op = np.diag(Uz)

    qc.gate(op, [i for i in range(num_qubits)])

    for i in range(num_qubits):

        qc.h(i)
        qc.p(-2*beta, i)
        qc.h(i)



#qc.draw()

In [48]:
qc.to_matrix()

array([[-0.00072327-7.63608301e-03j,  0.009846  +3.82786450e-03j,
         0.00696446-5.75464708e-03j, ..., -0.00250887+2.33970760e-03j,
        -0.00372461+4.65269789e-03j, -0.00204303-1.69936765e-03j],
       [ 0.00488685-1.85576418e-03j, -0.0007315 +7.15873578e-04j,
         0.0043101 -3.54678077e-03j, ...,  0.00406221-4.56790528e-03j,
         0.00396828+2.46609134e-03j,  0.00613295+1.52780936e-03j],
       [ 0.00895617-4.93120436e-03j,  0.00737681+2.46742795e-02j,
         0.00679449-1.15536879e-02j, ..., -0.00185472+1.18866646e-03j,
         0.00151323+3.95051245e-03j, -0.00033909+2.95715506e-03j],
       ...,
       [ 0.00027635-9.09034469e-04j, -0.00135225-9.25100170e-04j,
         0.00069297+6.34016552e-04j, ..., -0.00379365+4.80123709e-03j,
        -0.00282146-7.32101464e-03j, -0.00334134-5.51750427e-04j],
       [-0.0002377 -8.19184374e-04j, -0.00099469-3.99071339e-04j,
        -0.00058063-1.17185806e-03j, ..., -0.01396254+1.10087606e-02j,
         0.01195201+6.93376802e-03j

In [30]:
op

array([[-0.01034234-0.99994652j,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        , -0.32079639-0.94714818j,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.30113747-0.95358074j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       ...,
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ..., -0.59943745-0.8004216j ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
        -0.95664417-0.29125922j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j       

In [8]:
mps = QcMps(num_qubits, 0)
mps.run_from_qcirc(qc)

TypeError: apply_two_site_gate() takes 4 positional arguments but 12 were given

## Measure Loss
### in qiskit

In [14]:
import qiskit.quantum_info as qi
from qiskit.extensions.simulator import snapshot
from qiskit import Aer, transpile, QuantumRegister, AncillaRegister

In [39]:
class LossTracker:
    def __init__(self, num_qubits, num_ancillae):

        self.n_qubits   = num_qubits
        self.n_ancillae = num_ancillae
        self._statevecs = []
        self._h_perc    = None
        self._little_endian = True
        self._statevecs_arr = None


    def track(self, qc, compose=True):

        if compose:
            # create a copy of the current circuit internally
            self.current_qc = QuantumCircuit(QuantumRegister(self.n_qubits), AncillaRegister(self.n_ancillae))
            if len(self._statevecs) > 0:
                self.current_qc.initialize(self._statevecs[-1], range(self.n_qubits+self.n_ancillae))

            # compose the circuit
            if type(qc) is list:
                for circuit in qc:
                    self.current_qc = self.current_qc.compose(circuit.copy())
            elif type(qc) is QuantumCircuit:
                self.current_qc = self.current_qc.compose(qc.copy())
            else:
                print('Error: type of qc is', type(qc))
                return
        else:
            self.current_qc = qc

        # track the state
        self._statevecs.append(qi.Statevector.from_instruction(self.current_qc))
        del self.current_qc

    @property
    def statevecs(self):
        return self._statevecs
    
    @property
    def h_perc(self):
        return self._h_perc

    def reset(self, num_qubits=None, num_ancillae=None):
        self._statevecs.clear()
        self._statevecs_arr = None

        if num_qubits:
            self.n_qubits = num_qubits
        if num_ancillae:
            self.n_ancillae = num_ancillae

    def finalize(self):

        # convert statevectors to arrays, keep only qubits of interest
        arr_list = []
        for state in self._statevecs:
            out_red = qi.partial_trace(state, range(self.n_qubits, self.n_qubits + self.n_ancillae))
            prob, st_all = np.linalg.eig(out_red.data)
            idx    = np.argmax(prob) 
            arr_list.append(st_all[:, idx])

        self._statevecs_arr = np.stack(arr_list)
        del out_red, prob, st_all, idx, arr_list

    def __loss(self, statevec, h_perc):

        return np.vdot(statevec, h_perc * statevec)

    def get_losses(self, data, little_endian=True, labels=None):

        if len(self._statevecs) == 0:
            print('Error: no statevectors has been tracked down, please call track() before')
            return
        
        if labels is None:
            labels = np.ones((data.shape[0],))
        
        if self._statevecs_arr is None:
            print('LossTracker was not finalized, finalizing...')
            self.finalize()
            print('Done!')
            
        if self._h_perc is None or self._little_endian != little_endian:

            if little_endian:
                self._h_perc = H_perc_diag(data, labels)
            else:
                # invert data components if the circuit was constructed in big endian mode
                # NOT SURE IF THIS WORKS
                self._h_perc = H_perc_diag(data[:,::-1], labels)
           
           
        return np.apply_along_axis(self.__loss, axis=1, arr=self._statevecs_arr, h_perc=self._h_perc)
    
    def get_edensity(self, data, little_endian=True, labels=None):

        losses = self.get_losses(data, little_endian=little_endian, labels=labels)
        e0 = np.sort(self._h_perc)[0]

        return (losses-e0) / data.shape[1]


In [27]:
np.ones((3, )) * np.array([1, 2, 3])

array([1., 2., 3.])

In [21]:
N_feat = 4
N_data = 3

data = np.array([[1., 1., 1., 1.], 
                 [1., -1., 1., -1.], 
                 [1., -1., 1., 1.]])

labels = np.ones((N_data, ))

h_perc = H_perc_nobatch(data, labels)

def loss(statevec, h_perc):

    return np.vdot(statevec, np.dot(h_perc, statevec))

def get_losses_from_results(results, data, labels, num_qubits, representation='same'):
    statevectors = np.stack(results.data()['snapshots']['statevector']['weight_statevector'])[:,:2**num_qubits]

    if representation != 'same':
        # invert data components if the circuit was constructed in big endian mode
        # NOT SURE IF THIS WORKS
        h_perc = H_perc_nobatch(data[:,::-1], labels)
    else:
        h_perc = H_perc_nobatch(data, labels)

    return np.apply_along_axis(loss, axis=1, arr=statevectors, h_perc=h_perc)

def get_losses_from_sts(statevecs, data, labels, representation='same'):
    statevectors = np.stack(statevecs)

    if representation != 'same':
        # invert data components if the circuit was constructed in big endian mode
        # NOT SURE IF THIS WORKS
        h_perc = H_perc_nobatch(data[:,::-1], labels)
    else:
        h_perc = H_perc_nobatch(data, labels)

    return np.apply_along_axis(loss, axis=1, arr=statevectors, h_perc=h_perc)


Constructing H_perc:   0%|          | 0/3 [00:00<?, ?it/s]

In [47]:
def test():
    return QuantumCircuit(2)

In [50]:
test()

In [160]:
eigvals, eigvecs = np.linalg.eigh(h_perc)

print('min loss:', loss(eigvecs[:,0], h_perc), eigvals[0], '\nmax loss:', loss(eigvecs[:,-1], h_perc), eigvals[-1])

min loss: 0j 0.0 
max loss: (4+0j) 4.0


In [162]:
inspect = 6
eigvecs[:,inspect], eigvals[inspect]

(array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 1.0)

In [12]:
def your_evolution(p, num_qubits, num_ancillae):
    
    # implement yout evolution here
    qc = QuantumCircuit(num_qubits+num_ancillae)
    if p == 1:
        qc.h(range(num_qubits))
    if p == 2:
        qc.h(range(num_qubits))
        qc.x(range(num_qubits, num_qubits+num_ancillae))
    if p == 3:
        qc.x( 3)

    if p == 4:
        qc.x(2)

    return qc
    
def get_statevec(qc, num_qubits):

    statevec = qi.Statevector.from_instruction(qc)
    out_red = qi.partial_trace(statevec, range(num_qubits, qc.num_qubits))
    prob, st_all = np.linalg.eig(out_red.data)
    cond = np.argmax(prob)
    st = st_all[:, cond]
    
    return(st)


def get_statevec2(qc, num_qubits):
    qc.snapshot("weight_statevector", qubits=range(num_qubits))



In [40]:
N_data = 3
N_feat = 4
P = 3

num_qubits   = 4
num_ancillae = 2

qc_tot = QuantumCircuit(num_qubits + num_ancillae)
myst = []

loss_tracker = LossTracker(num_qubits, num_ancillae)

# apply evoltion 
for p in range(P):

    qc = your_evolution(p+1, num_qubits, num_ancillae)
    loss_tracker.track(qc)
    qc_tot = qc_tot.compose(qc)
    #get_statevec2(qc, num_qubits)


qc_tot.draw()

┌───┐┌───┐     
q_0: ┤ H ├┤ H ├─────
     ├───┤├───┤     
q_1: ┤ H ├┤ H ├─────
     ├───┤├───┤     
q_2: ┤ H ├┤ H ├─────
     ├───┤├───┤┌───┐
q_3: ┤ H ├┤ H ├┤ X ├
     ├───┤└───┘└───┘
q_4: ┤ X ├──────────
     ├───┤          
q_5: ┤ X ├──────────
     └───┘

In [41]:
loss_tracker.get_edensity(data)

LossTracker was not finalized, finalizing...
Done!


Constructing H_perc:   0%|          | 0/3 [00:00<?, ?it/s]

array([2.81250000e-01+0.j, 8.21166447e-33+0.j, 2.50000000e-01+0.j])

In [26]:
simulator = Aer.get_backend('aer_simulator_statevector')
circ = transpile(qc, simulator)

# Run and get counts
result = simulator.run(circ).result()

In [27]:
get_losses_from_results(result, data, labels, num_qubits, representation='same')

Constructing H_perc:   0%|          | 0/3 [00:00<?, ?it/s]

array([1.1250000e+00+0.j, 2.0884012e-32+0.j, 0.0000000e+00+0.j])

In [8]:
get_losses_from_sts(myst, data, labels, representation='same')

Constructing H_perc:   0%|          | 0/3 [00:00<?, ?it/s]

array([1.12500000e+00+0.j, 6.36949476e-34+0.j, 1.00000000e+00+0.j])

In [220]:
statevectors = np.stack(result.data()['snapshots']['statevector']['weight_statevector'])[:,:2**num_qubits]

In [192]:
last = statevectors[-1].copy()
last

array([-1.11022302e-16+6.12323400e-17j, -9.81307787e-18-6.16297582e-33j,
       -3.74939946e-33-6.16297582e-33j, -9.81307787e-18+3.08148791e-33j,
       -3.92523115e-17+6.16297582e-33j, -9.81307787e-18-2.29584502e-49j,
       -3.42113883e-49-2.29584502e-49j, -9.81307787e-18-6.84227766e-49j,
        1.00000000e+00-2.44929360e-16j, -9.81307787e-18+6.12323400e-17j,
        7.49879891e-33+6.12323400e-17j, -9.81307787e-18+8.55284707e-49j,
       -3.92523115e-17+6.12323400e-17j, -9.81307787e-18+2.29584502e-49j,
       -3.74939946e-33+2.29584502e-49j, -9.81307787e-18+3.08148791e-33j])

In [ ]:
permutation = []

num_arr = np.array([2**i for i in range(num_qubits-1, -1, -1)])

permutation.append(0)
for i in range(num_qubits):
    permutation.append(num_arr[i])

    for j in range(2**i):
        permutation.append(np.sum(num_arr[:j+1]) + num_arr[i])

### in qmatcha